In [ ]:
#pip install seaborn

In [ ]:
import pandas as pd
import mysql.connector
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style='whitegrid')
%matplotlib inline

In [ ]:
db_config = {
    "host": "localhost",
    "user": "john",
    "password": "blueharb0r!",
    "database": "blueharbor"
}


In [ ]:
joinquery = """
select
    s.shipment_id, s.origin_country, s.destination_country, s.declared_value, s.weight_kg, s.incoterm, s.shipment_date,
    p.product_name, p.product_category,
    t.vat_rate, t.customs_duty_rate,
    i.invoice_id, i.invoice_value, i.vat_amount, i.customs_amount, i.tax_flag, i.invoice_date
from shipments s
join invoices i on s.shipment_id = i.shipment_id
join products p on s.product_id = p.product_id
left join tax_rates t on s.destination_country = t.country;
"""

In [ ]:
conn = mysql.connector.connect(**db_config)

df = pd.read_sql(joinquery, conn)
conn.close()

In [ ]:
#total VAT collected by dest country graph
vat_per_country = df.groupby('destination_country')['vat_amount'].sum().sort_values(ascending=False)
vat_per_country.plot(kind='bar', figsize=(10,5), title= 'Total VAT - Destination Country', color='blue')
plt.ylabel('VAT')
# plt.show()


In [ ]:

#total custom collected by dest country
customs_per_country = df.groupby('destination_country')['customs_amount'].sum().sort_values(ascending=False)
customs_per_country.plot(kind='bar', figsize=(10,5), title= 'Total Customs - Destination Country', color='red')
plt.ylabel('Customs')
# plt.show()

In [ ]:

#total invoice value according to product category
invoice_per_cat = df.groupby('product_category')['invoice_value'].sum().sort_values(ascending=False)
invoice_per_cat.plot(kind='bar', figsize=(10,5), title= 'Total Invoice Value - Product Category', color='green')
plt.ylabel('Invoice Value')
# plt.show()

In [ ]:
#avg vat value by product category
avg_vat_by_cat = df.groupby('product_category')['invoice_value'].mean().sort_values(ascending=False)
avg_vat_by_cat.plot(kind='bar', figsize=(10,5), title= 'Total Invoice Value - Product Category', color='purple')
plt.ylabel('Invoice Value')
# plt.show()

In [ ]:
df['vat_ratio'] = (df['vat_amount']/df['declared_value']) * 100

vat_eff = df.groupby('product_category')['vat_ratio'].mean().sort_values(ascending=False)
vat_eff.plot(kind='bar',figsize=(10,5), title='VAT Efficiency by Product Category', color = 'teal')
plt.ylabel('VAT/Declared Value Percentage')
# plt.show()

In [ ]:
from ipywidgets import interact, widgets, interactive_output 
from datetime import date

def plot_dashboard2(selected_country="All", selected_category="All"):
    data = df.copy()

    if selected_country != "All":
        data = data[data['destination_country']==selected_country]

    if selected_category != "All":
        data = data[data['product_category']==selected_category]

    fig,axes = plt.subplots(2,2,figsize=(15,10))

    #total VAT collected by dest country graph
    vat_per_country = data.groupby('destination_country')['vat_amount'].sum()
    vat_per_country.plot(kind='bar', ax=axes[0,0], color='skyblue', title= 'Total VAT - Destination Country')
    axes[0,0].set_ylabel('VAT')
    

    #total custom collected by dest country
    customs_per_country = data.groupby('destination_country')['customs_amount'].sum()
    customs_per_country.plot(kind='bar', ax=axes[0,1], title= 'Total Customs - Destination Country', color='red')
    axes[0,1].set_ylabel('Customs')


    #total invoice value according to product category
    invoice_per_cat = data.groupby('product_category')['invoice_value'].sum()
    invoice_per_cat.plot(kind='bar', ax=axes[1,0], title= 'Total Invoice Value - Product Category', color='green')
    axes[1,0].set_ylabel('Invoice Value')

    vat_eff = data.groupby('product_category')['vat_ratio'].mean()
    vat_eff.plot(kind='bar',ax=axes[1,1], title='VAT Efficiency by Product Category', color = 'teal')
    axes[1,1].set_ylabel('VAT / Declared Value Percentage')

    plt.tight_layout()
    plt.show()

In [ ]:
# collected data visualisation
country_options = ["All"] + sorted(df['destination_country'].unique().tolist())
category_options = ["All"] + sorted(df['product_category'].unique().tolist())

interact(plot_dashboard2, 
         selected_country=widgets.Dropdown(options=country_options, description="Country"),
         selected_category=widgets.Dropdown(options=category_options, description="Category"))


In [ ]:
# Incoterms trend

incoterm_customs = (df.groupby("incoterm")["customs_amount"].mean().sort_values(ascending=False)
)
incoterm_customs.plot(kind="bar", color="coral", figsize=(8,4))
plt.title("Average Customs Cost by Incoterm")
plt.ylabel("Average Customs (£)")
plt.xlabel("Incoterm")
plt.tight_layout()
plt.show()

incoterm_vat = (df.groupby("incoterm")["vat_amount"].mean().sort_values(ascending=False)
)
incoterm_vat.plot(kind="bar", color="blue", figsize=(8,4))
plt.title("Average VAT Cost by Incoterm")
plt.ylabel("Average VAT (£)")
plt.xlabel("Incoterm")
plt.tight_layout()
plt.show()